In [1]:
# import packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline 
plt.style.use('seaborn')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [3]:
order_products_total = pd.concat([order_products_prior, order_products_train])

order_products_total.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [4]:
order_products_total.shape

(33819106, 4)

In [5]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
products.shape

(49688, 4)

In [8]:
df = order_products_total.merge(products, left_on='product_id', right_on='product_id', how='left')
df = df.merge(orders, left_on='order_id', right_on='order_id', how='left')
df = df.loc[:, ['user_id', 'product_id', 'product_name']]
df.head()

,user_id,product_id,product_name
0,202279,33120,Organic Egg Whites
1,202279,28985,Michigan Organic Kale
2,202279,9327,Garlic Powder
3,202279,45918,Coconut Butter
4,202279,30035,Natural Sweetener


In [9]:
df['present'] = 1

In [10]:
df.head()

,user_id,product_id,product_name,present
0,202279,33120,Organic Egg Whites,1
1,202279,28985,Michigan Organic Kale,1
2,202279,9327,Garlic Powder,1
3,202279,45918,Coconut Butter,1
4,202279,30035,Natural Sweetener,1


In [11]:
df.shape

(33819106, 4)

In [12]:
#pivot_table = df.pivot_table(index = 'user_id', columns='product_id', values = 'present').fillna()

In [13]:
from scipy.sparse import csr_matrix

users = list(df.user_id.unique())
products = list(df.product_id.unique())

In [14]:
users.sort()
len(users)

206209

In [15]:
products.sort()
len(products)

49685

In [16]:
data = df['present'].tolist()

In [17]:
len(data)

33819106

In [18]:
row = df.user_id.astype('category', categories=users).cat.codes
col = df.product_id.astype('category', categories=products).cat.codes

/Users/ka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.
/Users/ka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


In [19]:
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(products)))

In [20]:
#sparse_matrix.todense()

In [21]:
sparse_matrix.shape

(206209, 49685)

In [22]:
type(sparse_matrix)

scipy.sparse.csr.csr_matrix

In [23]:
u = list(range(1, 206209+1))
p = list(range(1, 49685+1))

In [24]:
sp_df = pd.DataFrame(sparse_matrix.todense(), index=u, columns=p)

In [25]:
sp_df.head()

,1,2,3,4,5,6,7,8,9,10,...,49676,49677,49678,49679,49680,49681,49682,49683,49684,49685
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
testing = sp_df.iloc[0:5,:]

In [28]:
user_means = testing.mean(axis=0)

In [30]:
df_pivot_demeaned = testing - user_means
df_pivot_demeaned.fillna(0, inplace=True)
df_pivot_demeaned.head()

,1,2,3,4,5,6,7,8,9,10,...,49676,49677,49678,49679,49680,49681,49682,49683,49684,49685
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0


In [44]:
df_pivot_demeaned

,1,2,3,4,5,6,7,8,9,10,...,49676,49677,49678,49679,49680,49681,49682,49683,49684,49685
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0


In [43]:
cosine_dists

,1,2,3,4,5
1,1.000000,-0.417685,-0.342749,-0.149789,-0.194872
2,-0.417685,1.000000,-0.336964,-0.426076,-0.400244
3,-0.342749,-0.336964,1.000000,-0.132091,-0.187347
4,-0.149789,-0.426076,-0.132091,1.000000,0.468098
5,-0.194872,-0.400244,-0.187347,0.468098,1.000000


In [42]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_dists = cosine_similarity(df_pivot_demeaned)

cosine_dists

cosine_dists = pd.DataFrame(cosine_dists, columns=df_pivot_demeaned.index)

cosine_dists.index = cosine_dists.columns
cosine_dists.iloc[0:5, :]

,1,2,3,4,5
1,1.000000,-0.417685,-0.342749,-0.149789,-0.194872
2,-0.417685,1.000000,-0.336964,-0.426076,-0.400244
3,-0.342749,-0.336964,1.000000,-0.132091,-0.187347
4,-0.149789,-0.426076,-0.132091,1.000000,0.468098
5,-0.194872,-0.400244,-0.187347,0.468098,1.000000


In [60]:
# create a new customer 6
# who like 5678 and 9999
# reshape and calculate cosine similarity with the first customer, and so on 
test_row = np.array([0] * len(df_pivot_demeaned.columns))
test_row[5678] =1
test_row[9999] = 1
test_row.reshape(1, -1).shape
df_pivot_demeaned.iloc[0, :].values
cosine_similarity(test_row.reshape(1, -1), df_pivot_demeaned.iloc[0, :].values.reshape(1, -1))

array([[0.]])

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_dists = cosine_similarity(df_pivot_demeaned)

cosine_dists

cosine_dists = pd.DataFrame(cosine_dists, columns=df_pivot_demeaned.index)

cosine_dists.index = cosine_dists.columns
cosine_dists.iloc[0:5, :]

grocery_i_like = 

grocery_summed = np.sum(cosine_dists[grocery_i_like], axis=1)

grocery_summed = grocery_summed.sort_values(ascending=False)
grocery_summed

ranked_grocery = grocery_summed.index[grocery_summed.isin(grocery_i_like)==False]
ranked_grocery = ranked_grocery.tolist()
ranked_grocery[:5]

KeyError: "['24852' '35073'] not in index"

In [ ]:
df_pivot.shape

df_pivot.iloc[0:5,0:5]

user_means = df_pivot.mean(axis=0)

df_pivot_demeaned = df_pivot - user_means
df_pivot_demeaned.fillna(0, inplace=True)
df_pivot_demeaned.head()

### Calculate distance between games

from sklearn.metrics.pairwise import cosine_similarity

cosine_dists = cosine_similarity(df_pivot_demeaned)

cosine_dists

cosine_dists = pd.DataFrame(cosine_dists, columns=df_pivot_demeaned.index)

cosine_dists.index = cosine_dists.columns
cosine_dists.iloc[0:5, 0:5]

games_i_like = ["Baldur's Gate II: Shadows of Amn", "BioShock"]

games_summed = np.sum(cosine_dists[games_i_like], axis=1)

games_summed = games_summed.sort_values(ascending=False)
games_summed

ranked_games = games_summed.index[games_summed.isin(games_i_like)==False]
ranked_games = ranked_games.tolist()
ranked_games[:5]

def get_similar_games(games_list, n=None):
    """
    calculates which beers are most similar to the inputs. Must not return
    the beers that were inputted.
    
    Parameters
    ----------
    beers: list
        some beers!
    
    Returns
    -------
    ranked_beers: list
        rank ordered beers
    """
    
    games = [game for game in games_list if game in cosine_dists.columns]
    games_summed = cosine_dists[games_list].apply(lambda row: np.sum(row), axis=1)
    games_summed = games_summed.sort_values(ascending=False)
    
    ranked_games = games_summed.index[games_summed.isin(games_list)==False]
    ranked_games = ranked_games.tolist()
    
    if n is None:
        return ranked_games
    else:
        return ranked_games[:n]

for i, game in enumerate(get_similar_games(["Baldur's Gate II: Shadows of Amn"], 10)):
    print("%d. %s on %s" % (i+1, game[0], game[1]))